In [1]:
simple_text=["Good Job Done","Very Good Job Done","Bad Job Done",
             "Very Bad Job Done","Bad Job"]
print(simple_text)

['Good Job Done', 'Very Good Job Done', 'Bad Job Done', 'Very Bad Job done', 'Bad Job']


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer()
vect.fit(simple_text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [16]:
#Created Dictionary from your data i.e. simple_text
#and Dictionary is nothing but feature columns
a=vect.get_feature_names()
print(a)

['bad', 'done', 'good', 'job', 'very']


In [15]:
class mycountvect:
    row=[]
    def my_fit(s,my_data):
        for i in my_data:
            a=i.split(' ')
            for j in a:
                j=j.lower()
                if (j in row):
                    continue
                else:
                    row.append(j)
        s.row=row
        
    def my_get_feature_names(s):
        return s.row
    
    def mytransform(s,my_data):
        #Create a  matrix- exercise

a=mycountvect()       
a.my_fit(simple_text)
print(a.my_get_feature_names())

['good', 'job', 'done', 'very', 'bad']


In [18]:
a=vect.get_feature_names()
print(a)

['bad', 'done', 'good', 'job', 'very']


In [17]:
#Create Matrix from data
data_matrix=vect.transform(simple_text)
print(data_matrix)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (1, 1)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (2, 0)	1
  (2, 1)	1
  (2, 3)	1
  (3, 0)	1
  (3, 1)	1
  (3, 3)	1
  (3, 4)	1
  (4, 0)	1
  (4, 3)	1


In [19]:
dense_matrix=data_matrix.toarray()
print(dense_matrix)

[[0 1 1 1 0]
 [0 1 1 1 1]
 [1 1 0 1 0]
 [1 1 0 1 1]
 [1 0 0 1 0]]


In [20]:
import pandas as pd
df=pd.DataFrame(dense_matrix,columns=vect.get_feature_names())
print(df)

   bad  done  good  job  very
0    0     1     1    1     0
1    0     1     1    1     1
2    1     1     0    1     0
3    1     1     0    1     1
4    1     0     0    1     0


In [21]:
# Come to the real problem 
email=pd.read_csv("sms.txt",header=None,names=['Label','Message'],sep='\t')
email.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [22]:
email.shape

(5572, 2)

In [23]:
email.Label.value_counts()

ham     4825
spam     747
Name: Label, dtype: int64

In [24]:
X=email.Message
y=email['Label']

In [25]:
#Split X and y in training and testing data set
# When test_size is not given , by default it is 0.25
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [31]:
X_train.shape

(4179,)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer()
vect.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [29]:
X_train_matrix=vect.transform(X_train)

In [30]:
X_train_matrix

<4179x7417 sparse matrix of type '<class 'numpy.int64'>'
	with 55855 stored elements in Compressed Sparse Row format>

In [ ]:
#that means your dictionary has 7417 unique words which exists
#in X_train messages

In [32]:
X_test_matrix=vect.transform(X_test)

In [33]:
X_test_matrix

<1393x7417 sparse matrix of type '<class 'numpy.int64'>'
	with 16905 stored elements in Compressed Sparse Row format>

In [34]:
# this process is known as Vectorization of data
#apply ML Algorithm

from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
nb.fit(X_train_matrix,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [35]:
# Model nb is created
y_predict=nb.predict(X_test_matrix)

In [36]:
print(y_predict)

['ham' 'ham' 'ham' ... 'ham' 'spam' 'ham']


In [37]:
s=nb.score(X_test_matrix, y_test)
print(s)

0.9863603732950467


In [39]:
from sklearn.metrics import accuracy_score,confusion_matrix
s1=accuracy_score(y_test,y_predict)
print(s1)

0.9863603732950467


In [41]:
print(confusion_matrix(y_test,y_predict))

[[1191    6]
 [  13  183]]


In [42]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train_matrix,y_train)
print(lr.score(X_test_matrix, y_test))


0.9791816223977028


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [44]:
#print message text for the false positive(FP)
#FP means: messages which are ham but incorrectly classifed as spam

X_test[(y_predict=='spam') & (y_test=="ham")]

1506    Total video converter free download type this ...
5475    Dhoni have luck to win some big title.so we wi...
4862                               Nokia phone is lovly..
3890                    Unlimited texts. Limited minutes.
4600              Have you laid your airtel line to rest?
4729    I (Career Tel) have added u as a contact on IN...
Name: Message, dtype: object

In [45]:
#print message text for the false negative(FN)
#FN means: messages which are spam but incorrectly classifed as ham

X_test[(y_predict=='ham') & (y_test=="spam")]

869     Hello. We need some posh birds and chaps to us...
2402    Babe: U want me dont u baby! Im nasty and have...
2558    This message is brought to you by GMW Ltd. and...
4949    Hi this is Amy, we will be sending you a free ...
3742                                        2/2 146tf150p
955             Filthy stories and GIRLS waiting for your
1500    SMS. ac JSco: Energy is high, but u may not kn...
1430    For sale - arsenal dartboard. Good condition b...
1875    Would you like to see my XXX pics they are so ...
3425    Am new 2 club & dont fink we met yet Will B gr...
3132    LookAtMe!: Thanks for your purchase of a video...
3530    Xmas & New Years Eve tickets are now on sale f...
4676    Hi babe its Chloe, how r u? I was smashed on s...
Name: Message, dtype: object

In [46]:
print(X[4949])

Hi this is Amy, we will be sending you a free phone number in a couple of days, which will give you an access to all the adult parties...
